In [2]:
from jsonobject import *
import numpy as np 
import random
import math

In [2]:
random.seed(2021)

# Classes

In [4]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [5]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)
    #shape = StringProperty()

In [6]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="'valid'")
    stride = IntegerProperty(default=None)

In [7]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [8]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

# Test

In [9]:
conv1 = Convolution(kernel=7, padding="valid", stride=2, nb_filter=6, fct_activation="tanh")
conv1.to_json()

{'kernel': 7,
 'padding': 'valid',
 'stride': 2,
 'nb_filter': 6,
 'fct_activation': 'tanh'}

In [10]:
inputL = InputLayer(shape=[28,28,1])
inputL.to_json()

{'shape': [28, 28, 1]}

In [11]:
pool1 = Pooling(op = "avg")
pool1.to_json()

{'op': 'avg', 'kernel': 2, 'padding': "'valid'", 'stride': None}

# Values

In [12]:
layers_name = [Convolution, InputLayer, Pooling, Flatten, Dense]

# Table of hyperparameter value
kernel_value = [1, 2, 3, 4, 5, 6, 7]
stride_value = [1, 2, 3]
padding_value = ["valid", "same"]
#nb_filter_value = [6, 16, 64, 128, 256, 512, 1024, 2048]
fct_activation_value = ["tanh", "relu", "selu"]


nb_class = 10 # nb_class
op_value = ['avg',"max"]

# Functions

In [13]:
def stop(size_archi, x):
    prob = x*size_archi-1*x
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [14]:
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    archi_file = open(directory+file_name, "a") # Open file in writting (a --> append)
    archi_file.write("""[
    """)
    archi_size = len(archi)
    i = 0
    for l in archi:

        str_layer = """\t{
            'class':'"""
        str_layer += l.__class__.__name__
        str_layer +="""',\n\t\t\t'parameters':"""
        str_layer += str(l.to_json())
        str_layer += """\n\t\t}"""
        if(i < archi_size-1):
            str_layer += ""","""
            i+=1
        str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

In [15]:
# compute the output 
# valid vs same
def calcul_output(input_size, l):
    output_size = 0
    if(l.padding == "valid"):
        kernel = l.kernel
        stride = l.stride
        while(input_size>=kernel):
            input_size -= stride
            output_size += 1   
    else:
        stride = l.stride
        if(input_size%stride == 0):
            output_size = int(input_size/stride)
        else:
            output_size = int(input_size/stride)+1
    return output_size

In [25]:
# add layer to architecture 

def addLayer(archi, layer):
    input_size = archi[0].shape[0]
    feature_extra = archi[1:]
    
    # add layer if the architecture is empty
    if(feature_extra == []):
        archi.append(layer)
        return 1, archi, calcul_output(input_size,layer)
    
    else :
        # compute size of the output of the last layer
        for l in feature_extra: 
            output_size = calcul_output(input_size, l)
            input_size = output_size
        
        # if we couldn't reduce more
        if(input_size == 1):
            return 0, archi, input_size
        
        # if the output size got more than 1 we can add new layer
        elif(output_size > 1): 
            output_size = calcul_output(input_size, layer)
            
            # if output size got negate is that the layer we want to add is wrong
            if(output_size < 1 ):
                return -1, archi, input_size
            # if output size is bigger than 0 we can add new layer and continue
            elif(output_size > 0):
                archi.append(layer)
                return 1, archi, output_size
            # this should not append
            else:
                return "Somethink wrong"
    # this should not append
    return "Error"

In [17]:
# test to understant padding

#from keras.models import Sequential
#from keras.layers import Conv2D, MaxPool2D, AveragePooling2D

#model = Sequential(layers=[
#    Conv2D(6, 5, input_shape=(28, 28, 1), strides=1, padding="same"),
#    MaxPool2D(pool_size=2, strides=2, padding="same"),
#    
#])

#for layer in model.layers:
#    print(layer.output_shape)

In [27]:
# test for the fonction addLayer()
architecture = list()
architecture.append(InputLayer(shape=[29,29,1]))



code, architecture, out = addLayer(architecture.copy(), conv1)

print("out : " + str(out))

print("code : " + str(code))

code, architecture, out = addLayer(architecture.copy(), conv1)

print("out : " + str(out))

print("code : " + str(code))

code, architecture, out = addLayer(architecture.copy(), conv1)

print("out : " + str(out))

print("code : " + str(code))

architecture


out : 12
code : 1
out : 3
code : 1
out : 3
code : -1


[InputLayer(shape=[29, 29, 1]),
 Convolution(fct_activation='tanh', kernel=7, nb_filter=6, padding='valid', stride=2),
 Convolution(fct_activation='tanh', kernel=7, nb_filter=6, padding='valid', stride=2)]

# Creation architecture

In [19]:
extraction_feature= [Pooling, Convolution]

In [37]:
for i in range (1,11):
    architecture = list() # init list
    
    # add Input layer
    architecture.append(InputLayer(shape=[32,32,3]))
    
    nb_filter_value = 6*3 # init nb feature map
    output_size = 32   
    
    # add extraction feature (succession of Pooling/convolution)
    # Pooling can't be follow by a Pooling
    pooling = True
    code = 1 # we can add new layer
    j = 2
    while((output_size == 32) | (stop(len(architecture),5) & code == 1)):
        layer = extraction_feature[random.randrange(2)]
        
                
        kernel=kernel_value[random.randrange(7)]
        stride_value_filtered = [value for value in stride_value if value <= kernel]
        stride = stride_value_filtered[random.randrange(len(stride_value_filtered))]
        
        if(pooling | isinstance(layer, Convolution)):
            add_layer = Convolution(
                    kernel=kernel, 
                    padding=padding_value[random.randrange(2)], 
                    stride=stride, 
                    nb_filter= nb_filter_value,
                    fct_activation=fct_activation_value[random.randrange(3)]
            )
        else: 
            add_layer = Pooling(
                op = op_value[random.randrange(2)],
                kernel=kernel, 
                padding=padding_value[random.randrange(2)], 
                stride=stride
            )
        code, architecture, output_size = addLayer(architecture, add_layer)
        if(code == 1 & isinstance(add_layer, Convolution)):
            pooling = False
            nb_filter_value = nb_filter_value*j
        else: pooling = True
        
    
    # add flatten Layer
    
    print("output size : " + str(output_size))
    print("nb_filter : " + str(nb_filter_value/2))
    print("code: " + str(code))
    
    architecture.append(Flatten())

    param = output_size*output_size*(nb_filter_value/2)
    print('param : ' + str(param))
    
    #nb_layer = random.randrange(1,10)
   
    #print("nb_layer: "+ str(nb_layer))
    
    # add Dense Layer
    
    ## init values
    pourcent = random.uniform(10,90)
    nb=0
    nb_neurones = int(param*pourcent/100)
    
    print("nb_neurone : " + str(nb_neurones))
    while(nb_neurones > nb_class):
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))
        
        architecture.append(Dense(
                nb_neurones = nb_neurones,
                fct_activation = fct_activation_value[random.randrange(3)]
        ))
        
        #incrementation
        pourcent = random.uniform(10,90)
        nb+=1
        nb_neurones = int(nb_neurones*pourcent/100)

        
        
    
    architecture.append(Dense(
        nb_neurones = nb_class,
        fct_activation="softmax"           
    ))
                        
                        
    # print architecture
    print(architecture)
    
    # create file
    file_name = "archi_random_%d_v.json" % i
    print(file_name)
    print("")
    create_json_file(architecture, file_name)

output size : 3
nb_filter : 36.0
code: 1
param : 324.0
nb_neurone : 252
=====
iteration0
nb_neurone : 252
=====
iteration1
nb_neurone : 215
=====
iteration2
nb_neurone : 127
=====
iteration3
nb_neurone : 67
=====
iteration4
nb_neurone : 18
=====
iteration5
nb_neurone : 15
[InputLayer(shape=[32, 32, 3]), Convolution(fct_activation='selu', kernel=1, nb_filter=18, padding='same', stride=1), Pooling(kernel=6, op='avg', padding='valid', stride=3), Convolution(fct_activation='tanh', kernel=6, nb_filter=36, padding='same', stride=3), Flatten(), Dense(fct_activation='selu', nb_neurones=252), Dense(fct_activation='selu', nb_neurones=215), Dense(fct_activation='selu', nb_neurones=127), Dense(fct_activation='relu', nb_neurones=67), Dense(fct_activation='relu', nb_neurones=18), Dense(fct_activation='tanh', nb_neurones=15), Dense(fct_activation='softmax', nb_neurones=10)]
archi_random_1_v.json

output size : 1
nb_filter : 144.0
code: 0
param : 144.0
nb_neurone : 110
=====
iteration0
nb_neurone : 11

# Idee ajout Resnet
- DSL
    - Ex: lenet --> ICACAFDDD
    - Ex: Resnet --> ICM[CC][CC][CC]AFDDD
    - Ex: Densenet --> ICM[[[CC][[CC]][CC]]]FDDD
    - Ex: Xception --> ICM[[[CC][CC]][CC]]FDDD
- Ensemble (Diagramme de venn)
![image](../Sanstitre.png)

In [3]:
e = math.e

In [4]:
math.log(2*10)*10

29.957322735539908